In [1]:
!uv add numpy==1.26

'uv' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error, mean_squared_error
from collections import defaultdict
import math
from tqdm import tqdm
from surprise import Dataset, Reader, SVD
from surprise import accuracy
import seaborn as sns
import re
import os 
#import isodate


In [3]:
# Load CSVs
recipes = pd.read_csv("recipes.csv", dtype={'CookTime': str, 'PrepTime': str, 'TotalTime': str})
reviews = pd.read_csv("reviews.csv")

print(recipes.shape, reviews.shape)
recipes.head()


(522517, 28) (1401982, 8)


,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ..."
3,41,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03T14:54:00Z,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc..."
4,42,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19T06:19:00Z,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,NaN,"c(""Mix everything together and bring to a boil..."


In [4]:
print(recipes.shape, reviews.shape)
reviews.head()

(522517, 28) (1401982, 8)


,ReviewId,RecipeId,AuthorId,AuthorName,Rating,Review,DateSubmitted,DateModified
0,2,992,2008,gayg msft,5,better than any you can get at a restaurant!,2000-01-25T21:44:00Z,2000-01-25T21:44:00Z
1,7,4384,1634,Bill Hilbrich,4,"I cut back on the mayo, and made up the differ...",2001-10-17T16:49:59Z,2001-10-17T16:49:59Z
2,9,4523,2046,Gay Gilmore ckpt,2,i think i did something wrong because i could ...,2000-02-25T09:00:00Z,2000-02-25T09:00:00Z
3,13,7435,1773,Malarkey Test,5,easily the best i have ever had. juicy flavor...,2000-03-13T21:15:00Z,2000-03-13T21:15:00Z
4,14,44,2085,Tony Small,5,An excellent dish.,2000-03-28T12:51:00Z,2000-03-28T12:51:00Z


In [5]:
# Drop duplicate recipes/reviews
recipes = recipes.drop_duplicates(subset="RecipeId")
reviews = reviews.drop_duplicates(subset="ReviewId")

nutritional_cols = [
    "Calories","FatContent","SaturatedFatContent","CholesterolContent",
    "SodiumContent","CarbohydrateContent","FiberContent","SugarContent","ProteinContent"
]
for col in nutritional_cols:
    recipes[col] = pd.to_numeric(recipes[col], errors="coerce")


In [6]:
n_users = reviews["AuthorId"].nunique()
n_items = reviews["RecipeId"].nunique()

# Total ratings
n_ratings = len(reviews)

# Average ratings
avg_ratings_per_user = n_ratings / n_users
avg_ratings_per_item = n_ratings / n_items
avg_rating = reviews["Rating"].mean()
rating_sd = reviews["Rating"].std()

ratings_per_user = reviews.groupby("AuthorId").size()
ratings_per_item = reviews.groupby("RecipeId").size()

max_ratings_by_user = ratings_per_user.max()
max_ratings_by_item = ratings_per_item.max()

# Density of ratings
density = n_ratings / (n_users * n_items)

print(density)

# Put it all into a summary table
summary = pd.DataFrame({
    "Number of users": [n_users],
    "Number of items": [n_items],
    "Total ratings": [n_ratings],
    "Avg ratings per user": [avg_ratings_per_user],
    "Avg ratings per item": [avg_ratings_per_item],
    "Avg rating": [avg_rating],
    "Rating SD": [rating_sd],
    "Max ratings by user": [max_ratings_by_user],
    "Max ratings by item": [max_ratings_by_item],
    "Density": [density]
})

display(summary.round(3))

1.897874882176401e-05


,Number of users,Number of items,Total ratings,Avg ratings per user,Avg ratings per item,Avg rating,Rating SD,Max ratings by user,Max ratings by item,Density
0,271907,271678,1401982,5.156,5.16,4.408,1.272,8842,2892,0.0


In [59]:
print("Number of recipes:", recipes["RecipeId"].nunique())
print("Number of authors:", recipes["AuthorId"].nunique())
print("Average rating:", recipes["AggregatedRating"].mean())
print("Rating SD:", recipes["AggregatedRating"].std())
print("Average review count per recipe:", recipes["ReviewCount"].mean())
print("Average calories:", recipes["Calories"].mean())
print("Calories SD:", recipes["Calories"].std())
print("Max review count for a recipe:", recipes["ReviewCount"].max())

# https://chatgpt.com/share/68e00f38-e5e8-8013-bc4c-246ca4534103
def parse_duration_auto(s):
    if pd.isna(s):
        return np.nan
    
    s = str(s).strip().lower()
    
    # Try ISO 8601 first
    try:
        td = isodate.parse_duration(s)
        return td.total_seconds() / 60  # convert to minutes
    except:
        pass
    
    # Try common text formats like '1 hr 30 min', '45 mins'
    hours = re.search(r'(\d+)\s*(h|hr|hour)', s)
    minutes = re.search(r'(\d+)\s*(m|min|minute)', s)
    total = 0
    if hours: total += int(hours.group(1)) * 60
    if minutes: total += int(minutes.group(1))
    
    # Only a number given, assume minutes
    if total == 0:
        num = re.search(r'\d+', s)
        if num:
            total = int(num.group(0))
    
    return total if total > 0 else np.nan

# Apply to columns
for col in ['CookTime', 'PrepTime', 'TotalTime']:
    recipes[col + '_minutes'] = recipes[col].apply(parse_duration_auto)

time_cols = ['CookTime_minutes', 'PrepTime_minutes', 'TotalTime_minutes']

for col in time_cols:
    # Remove NaN and extreme outliers (above 99th percentile)
    valid = recipes[col].dropna()
    upper = valid.quantile(0.99)
    filtered = valid[valid <= upper]
    
    plt.figure(figsize=(6,4))
    filtered.hist(bins=50)
    plt.xlabel(f"{col} (minutes)")
    plt.ylabel("Count")
    plt.title(f"Distribution of {col}")
    plt.savefig(os.path.join("plots", f"{col}_distribution.png"))
    plt.close()


# Calories stats
calories_valid = recipes["Calories"].replace(0, np.nan).dropna()
upper = calories_valid.quantile(0.99)
calories_filtered = calories_valid[calories_valid <= upper]

plt.figure(figsize=(6,4))
calories_filtered.hist(bins=50)
plt.xlabel("Calories")
plt.ylabel("Count")
plt.title("Distribution of Calories")
plt.savefig(os.path.join("plots", "calories_distribution.png"))
plt.close()

# Categories, truncate amount of categories for better visualization
recipes['RecipeCategory'] = recipes['RecipeCategory'].fillna('Unknown')
top_categories = recipes['RecipeCategory'].value_counts().nlargest(10).index
recipes['RecipeCategory'] = recipes['RecipeCategory'].apply(lambda x: x if x in top_categories else 'Other')
plt.figure(figsize=(6,4))
recipes['RecipeCategory'].value_counts().plot(kind='bar')
plt.xlabel("Recipe Category")
plt.ylabel("Count")
plt.title("Number of recipes per category")
plt.savefig(os.path.join("plots", "recipe_category_distribution.png"))
plt.close()

# Ingredients per recipe 
recipes['num_ingredients'] = recipes['RecipeIngredientParts'].apply(lambda x: len(str(x).split(',')) if pd.notna(x) else 0)
plt.figure(figsize=(6,4))
recipes['num_ingredients'].hist(bins=50)
plt.xlabel("Number of ingredients")
plt.ylabel("Count")
plt.title("Distribution of number of ingredients per recipe")
plt.savefig(os.path.join("plots", "num_ingredients_distribution.png"))
plt.close()

Number of recipes: 522517
Number of authors: 57178
Average rating: 4.632013709922984
Rating SD: 0.6419341051272117
Average review count per recipe: 5.227784080166383
Average calories: 484.4385799887851
Calories SD: 1397.116649087612
Max review count for a recipe: 3063.0


In [7]:
# Rating distribution
plt.figure(figsize=(6,4))
reviews["Rating"].hist(bins=np.arange(0.5, 6, 1), rwidth=0.8)
plt.xlabel("Rating")
plt.ylabel("Count")
plt.title("Distribución de Ratings")
plt.savefig(os.path.join("plots", "rating_distribution.png"))
plt.close()

# Numero de reseñas por usuario
user_counts = reviews.groupby("AuthorId").size()
plt.figure(figsize=(6,4))
user_counts.hist(bins=50, log=True)
plt.xlabel("Número de reseñas por usuario")
plt.ylabel("Frecuencia")
plt.title("Distribución de reseñas por usuario")
plt.savefig(os.path.join("plots", "user_review_distribution.png"))
plt.close()

item_counts = reviews.groupby("RecipeId").size()
plt.figure(figsize=(6,4))
item_counts.hist(bins=50, log=True)
plt.xlabel("Número de reseñas por receta")
plt.ylabel("Frecuencia")
plt.title("Distribución de reseñas por receta")
plt.savefig(os.path.join("plots", "item_review_distribution.png"))
plt.close()


In [8]:
# Split train/test
train, test = train_test_split(reviews, test_size=0.2)

# Random predictions
test["pred_random"] = np.random.randint(1, 6, size=len(test))

# Most Popular (item mean)
item_means = train.groupby("RecipeId")["Rating"].mean()
test["pred_item_mean"] = test["RecipeId"].map(item_means).fillna(train["Rating"].mean())

# User Mean
user_means = train.groupby("AuthorId")["Rating"].mean()
test["pred_user_mean"] = test["AuthorId"].map(user_means).fillna(train["Rating"].mean())

rmse_random = root_mean_squared_error(test["Rating"], test["pred_random"])
rmse_item = root_mean_squared_error(test["Rating"], test["pred_item_mean"])
rmse_user = root_mean_squared_error(test["Rating"], test["pred_user_mean"])

print("RMSE Random:", rmse_random)
print("RMSE Item mean:", rmse_item)
print("RMSE User mean:", rmse_user)

mae_random = np.mean(np.abs(test["Rating"] - test["pred_random"]))
mae_item = np.mean(np.abs(test["Rating"] - test["pred_item_mean"]))
mae_user = np.mean(np.abs(test["Rating"] - test["pred_user_mean"]))

print("MAE Random:", mae_random)
print("MAE Item mean:", mae_item)
print("MAE User mean:", mae_user)

RMSE Random: 2.3684797343654225
RMSE Item mean: 1.3382054817810864
RMSE User mean: 1.262977521343396
MAE Random: 1.9210547901725055
MAE Item mean: 0.8239708008207418
MAE User mean: 0.7266119648232167


In [9]:
def get_top_n(predictions, n=10): # Obtenida de https://surprise.readthedocs.io/en/stable/FAQ.html
    """Devuelve el top-N para cada usuario a partir de un conjunto de predicciones."""

    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Ordenar predicciones por score
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [10]:
popular_items = (
    train.groupby("RecipeId")["Rating"]
    .count()
    .sort_values(ascending=False)
    .index
    .tolist()
)

# Items seen by each user
user_items_train = train.groupby("AuthorId")["RecipeId"].apply(set).to_dict()
all_items = set(train["RecipeId"].unique())

user_means = train.groupby("AuthorId")["Rating"].mean().to_dict()
item_means = train.groupby("RecipeId")["Rating"].mean().to_dict()
global_mean = train["Rating"].mean()


In [11]:
all_items = np.array(list(train["RecipeId"].unique()))
shuffled_items = np.random.permutation(all_items)

def recommend_random(uid, n=10):
    seen = user_items_train.get(uid, set())
    recs = [iid for iid in shuffled_items if iid not in seen]
    return recs[:n]


def recommend_popular(uid, n=10):
    seen = user_items_train.get(uid, set())
    recs = [iid for iid in popular_items if iid not in seen]
    return recs[:n]

In [ ]:
def precision_recall_at_k(recs, relevant, k=10):
    recs_k = recs[:k]
    hits = len(set(recs_k) & set(relevant))
    precision = hits / k if k else 0
    recall = hits / len(relevant) if relevant else 0
    return precision, recall

def ndcg_at_k(recs, relevant, k=10):
    recs_k = recs[:k]
    dcg = sum(1 / math.log2(i+2) for i, iid in enumerate(recs_k) if iid in relevant)
    idcg = sum(1 / math.log2(i+2) for i in range(min(len(relevant), k)))
    return dcg / idcg if idcg > 0 else 0


# Map NutriScore letters to numeric values if needed, e.g. A=5, B=4, ..., E=1
nutri_map = {"A": 5, "B": 4, "C": 3, "D": 2, "E": 1}

def mean_health_at_k(top_n, recipes_df, k=10):
    all_scores = []
    for uid, recs in top_n.items():
        for iid, _ in recs[:k]:
            row = recipes_df.loc[recipes_df["RecipeId"] == iid]
            if row.empty:
                continue
            score_letter = row.iloc[0]["NutriScore"]
            score = nutri_map.get(score_letter, np.nan)
            if not np.isnan(score):
                all_scores.append(score)
    
    return np.mean(all_scores) if all_scores else np.nan



In [ ]:
# Negative points look-up tables
energy_points = [80,160,240,320,400,480,560,640,720,800]
sugar_points = [4.5,9,13.5,18,22.5,27,31,36,40,45]
sat_fat_points = [1,2,3,4,5,6,7,8,9,10]
salt_points = [90/400, 180/400, 270/400, 360/400, 450/400, 540/400, 630/400, 720/400, 810/400, 900/400]  

# Positive points tables
fiber_points = [0.7, 1.4, 2.1, 2.8, 3.5]
protein_points = [1.6, 3.2, 4.8, 6.4, 8.0]


def score_from_thresholds(value, thresholds):
    pts = 0
    for t in thresholds:
        if value > t:
            pts += 1
    return pts


def compute_nutriscore(row):
    # NEGATIVE POINTS 
    A = score_from_thresholds(row["Calories"], energy_points)
    B = score_from_thresholds(row["SugarContent"], sugar_points)
    C = score_from_thresholds(row["SaturatedFatContent"], sat_fat_points)

    salt = row["SodiumContent"] / 400  
    D = score_from_thresholds(salt, salt_points)

    N = A + B + C + D

    # POSITIVE POINTS
    # No fruit/veg/nut % available, assume median
    E = 1

    F = score_from_thresholds(row["FiberContent"], fiber_points)
    G = score_from_thresholds(row["ProteinContent"], protein_points)

    if N >= 11:
        G = 0

    P = E + F + G

    total = N - P
    if total <= -1:
        return "A"
    elif total <= 2:
        return "B"
    elif total <= 10:
        return "C"
    elif total <= 18:
        return "D"
    else:
        return "E"


recipes["NutriScore"] = recipes.apply(compute_nutriscore, axis=1)
recipes["NutriScore"].head(20)

0     B
1     E
2     D
3     E
4     C
5     E
6     E
7     C
8     A
9     B
10    E
11    E
12    A
13    A
14    B
15    D
16    E
17    C
18    E
19    A
Name: NutriScore, dtype: object

In [16]:
# Build test relevance dict (what items each user interacted with in test set)
user_items_test = test.groupby("AuthorId")["RecipeId"].apply(set).to_dict()

In [17]:
results = {"random": [], "popular": []}
k = 10

# Evaluate
for uid, relevant in tqdm(user_items_test.items(), desc="Evaluating users"):
    # Random
    recs_rand = recommend_random(uid, n=k)
    p, r = precision_recall_at_k(recs_rand, relevant, k)
    n = ndcg_at_k(recs_rand, relevant, k)
    results["random"].append((p, r, n))
    
    # Popular
    recs_pop = recommend_popular(uid, n=k)
    p, r = precision_recall_at_k(recs_pop, relevant, k)
    n = ndcg_at_k(recs_pop, relevant, k)
    results["popular"].append((p, r, n))


Evaluating users: 100%|██████████| 82946/82946 [29:03<00:00, 47.58it/s]


In [18]:
from surprise.model_selection import train_test_split
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(reviews[["AuthorId", "RecipeId", "Rating"]], reader)

trainset, testset = train_test_split(data, test_size=0.2)
algo = SVD()
algo.fit(trainset)

# Predict on test set
predictions = algo.test(testset)
rmse_svd = accuracy.rmse(predictions, verbose=True)
mae_svd = accuracy.mae(predictions, verbose=True)
print("RMSE SVD:", rmse_svd)

# Get recommendations
top_n = get_top_n(predictions, n=10)

# Example: print recommendations for first 3 users
for uid, recs in list(top_n.items())[:3]:
    print(f"Usuario {uid} -> {[iid for iid, _ in recs]}")


RMSE: 1.2252
MAE:  0.7447
RMSE SVD: 1.2251605265837497
Usuario 253710 -> [138062, 17726, 125487, 192408, 26769, 59397, 157364, 252245, 19367, 87075]
Usuario 211331 -> [13847, 304615, 216041, 280342, 294035, 81347, 346932, 23919, 161345, 446559]
Usuario 291568 -> [125859, 100198, 106251, 368428, 163547, 176694, 33257, 151458]


In [ ]:
def get_top_n(predictions, n=10):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

# Example: print recommendations for first 3 users
for uid, recs in list(top_n.items())[:3]:
    print(f"Usuario {uid} -> {[iid for iid, _ in recs]}")

Usuario 253710 -> [138062, 17726, 125487, 192408, 26769, 59397, 157364, 252245, 19367, 87075]
Usuario 211331 -> [13847, 304615, 216041, 280342, 294035, 81347, 346932, 23919, 161345, 446559]
Usuario 291568 -> [125859, 100198, 106251, 368428, 163547, 176694, 33257, 151458]


In [20]:
test_user_items = defaultdict(list)
for uid, iid in zip(test["AuthorId"], test["RecipeId"]):
    test_user_items[uid].append(iid)


In [21]:
def precision_recall_at_k(top_n, test_user_items, k=10):
    precisions = []
    recalls = []
    for uid, recs in top_n.items():
        if uid not in test_user_items:
            continue
        relevant = set(test_user_items[uid])
        recommended = set([iid for iid, _ in recs[:k]])
        precisions.append(len(recommended & relevant) / k)
        recalls.append(len(recommended & relevant) / len(relevant))
    return np.mean(precisions), np.mean(recalls)

precision, recall = precision_recall_at_k(top_n, test_user_items, k=10)
print(f"Precision@10: {precision:.4f}, Recall@10: {recall:.4f}")

def ndcg_at_k(top_n, test_user_items, k=10):
    ndcgs = []
    for uid, recs in top_n.items():
        if uid not in test_user_items:
            continue
        relevant = set(test_user_items[uid])
        dcg = 0
        for i, (iid, _) in enumerate(recs[:k]):
            if iid in relevant:
                dcg += 1 / math.log2(i + 2) 
        idcg = sum(1 / math.log2(i + 2) for i in range(min(k, len(relevant))))
        ndcgs.append(dcg / idcg if idcg > 0 else 0)
    return np.mean(ndcgs)

ndcg = ndcg_at_k(top_n, test_user_items, k=10)
print(f"NDCG@10: {ndcg:.4f}")


Precision@10: 0.0846, Recall@10: 0.4289
NDCG@10: 0.4324


In [ ]:
import pandas as pd
import torch
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from tqdm import tqdm

from deepctr_torch.inputs import SparseFeat, get_feature_names
from deepctr_torch.models import DeepFM

# 1. Load and prepare data
reviews_deep = pd.read_csv('reviews.csv')
recipes = pd.read_csv('recipes.csv')
reviews_deep.rename(columns={"AuthorId": "user", "RecipeId": "item", "Rating": "label"}, inplace=True)

# Reorder columns to have 'user' and 'item' as the first two columns
reviews_deep = reviews_deep[['user', 'item', 'label']]

sparse_features = ["user", "item"]
target = ['label']

# 2. Label Encoding for sparse features
user_lbe = LabelEncoder()
item_lbe = LabelEncoder()
reviews_deep['user'] = user_lbe.fit_transform(reviews_deep['user'])
reviews_deep['item'] = item_lbe.fit_transform(reviews_deep['item'])


# 3. Generate feature columns
fixlen_feature_columns = [SparseFeat(feat, reviews_deep[feat].nunique(), embedding_dim=16) for feat in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# 4. Split data into training and testing sets
train, test = train_test_split(reviews_deep, test_size=0.2, random_state=2020)
train_model_input = {name: train[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}

# 5. Define, compile, and train the DeepFM model
device = 'cpu'
if torch.cuda.is_available():
    print('CUDA is available. Using GPU.')
    device = 'cuda:0'
else:
    print('CUDA is not available. Using CPU.')

model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression', device=device)
model.compile("adam", "mse", metrics=['mae', 'mse'])

history = model.fit(train_model_input, train[target].values,
                    batch_size=2048, epochs=3, verbose=2, validation_split=0.2)

# 6. Evaluate the model for regression metrics
pred_ans = model.predict(test_model_input, batch_size=256)
mae = mean_absolute_error(test[target].values, pred_ans)
mse = mean_squared_error(test[target].values, pred_ans)
rmse = np.sqrt(mse)

print(f"Test MAE: {mae:.4f}")
print(f"Test RMSE: {rmse:.4f}")

# 7. Evaluate the model for ranking metrics
def get_top_n(predictions, n=10):
    top_n = {}
    for uid, iid, true_rating, pred_rating in predictions:
        if uid not in top_n:
            top_n[uid] = []
        top_n[uid].append((iid, pred_rating))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

def precision_recall_at_k(top_n, test_data, k=10):
    precisions = dict()
    recalls = dict()

    test_user_items = test_data.groupby('user')['item'].apply(list).to_dict()

    for uid, user_ratings in top_n.items():
        if uid in test_user_items:
            n_rel = len(test_user_items[uid])
            n_rec_and_rel = len([iid for iid, _ in user_ratings if iid in test_user_items[uid]])
            
            precisions[uid] = n_rec_and_rel / k
            recalls[uid] = n_rec_and_rel / n_rel if n_rel != 0 else 0

    return precisions, recalls

def ndcg_at_k(top_n, test_data, k=10):
    ndcgs = dict()
    
    test_user_items = test_data.groupby('user').apply(lambda x: dict(zip(x['item'], x['label']))).to_dict()

    for uid, user_ratings in top_n.items():
        if uid in test_user_items:
            dcg = 0
            idcg = 0
            
            # DCG
            for i, (iid, _) in enumerate(user_ratings):
                if iid in test_user_items[uid]:
                    relevance = test_user_items[uid][iid]
                    dcg += (2**relevance - 1) / np.log2(i + 2)
            
            # IDCG
            sorted_true_ratings = sorted(test_user_items[uid].values(), reverse=True)
            for i, rating in enumerate(sorted_true_ratings[:k]):
                idcg += (2**rating - 1) / np.log2(i + 2)

            ndcgs[uid] = dcg / idcg if idcg > 0 else 0
            
    return ndcgs

all_items = reviews_deep['item'].unique()
train_user_items = train.groupby('user')['item'].apply(list).to_dict()
test_users = test['user'].unique()

all_predictions = []
for user in tqdm(test_users):
    if user in train_user_items:
        user_interacted_items = train_user_items[user]
        items_to_predict = np.setdiff1d(all_items, user_interacted_items)
    else:
        items_to_predict = all_items

    user_repeated = np.full(len(items_to_predict), user)
    
    prediction_data = {'user': user_repeated, 'item': items_to_predict}
    
    pred_ratings = model.predict(prediction_data, batch_size=2048)
    
    for i, item in enumerate(items_to_predict):
        true_rating_series = test[(test['user'] == user) & (test['item'] == item)]['label']
        true_rating = true_rating_series.iloc[0] if not true_rating_series.empty else 0
        all_predictions.append([user, item, true_rating, pred_ratings[i]])

top_n = get_top_n(all_predictions, n=10)
precisions, recalls = precision_recall_at_k(top_n, test, k=10)
ndcgs = ndcg_at_k(top_n, test, k=10)

avg_precision = np.mean(list(precisions.values()))
avg_recall = np.mean(list(recalls.values()))
avg_ndcg = np.mean(list(ndcgs.values()))

print(f"Precision@10: {avg_precision:.4f}")
print(f"Recall@10: {avg_recall:.4f}")
print(f"NDCG@10: {avg_ndcg:.4f}")

nutri_map = {"A": 5, "B": 4, "C": 3, "D": 2, "E": 1}

def mean_health_at_k(top_n, recipes_df, item_lbe, k=10):
    all_scores = []
    for uid, recs in top_n.items():
        for iid_encoded, _ in recs[:k]:
            try:
                # Inverse transform the item id to its original value
                iid_original = item_lbe.inverse_transform([iid_encoded])[0]
                row = recipes_df.loc[recipes_df["RecipeId"] == iid_original]
                if row.empty:
                    continue
                score_letter = row.iloc[0]["NutriScore"]
                score = nutri_map.get(score_letter, np.nan)
                if not np.isnan(score):
                    all_scores.append(score)
            except IndexError:
                pass
    
    return np.mean(all_scores) if all_scores else np.nan

mean_health = mean_health_at_k(top_n, recipes, item_lbe)
print(f"MeanHealth@10: {mean_health}")

In [ ]:
# Create top_n for Random recommendations
top_n_random = {}
for uid in tqdm(user_items_test.keys(), desc="Generating Random recommendations"):
    recs = recommend_random(uid, n=10)
    top_n_random[uid] = [(iid, 0) for iid in recs] 

# Create top_n for Popular recommendations
top_n_popular = {}
for uid in tqdm(user_items_test.keys(), desc="Generating Popular recommendations"):
    recs = recommend_popular(uid, n=10)
    top_n_popular[uid] = [(iid, 0) for iid in recs] 

# Create top_n for User Mean recommendations
top_n_user_mean = {}
test_user_groups = test.groupby('AuthorId')
for uid, group in tqdm(test_user_groups, desc="Generating User Mean recommendations"):
    # Sort items by the predicted user mean rating
    top_items = group.sort_values('pred_user_mean', ascending=False).head(10)
    top_n_user_mean[uid] = list(zip(top_items['RecipeId'], top_items['pred_user_mean']))

Generating User Mean recommendations: 100%|██████████| 82946/82946 [00:21<00:00, 3913.57it/s]


In [27]:
mean_health_random = mean_health_at_k(top_n_random, recipes, k=10)
mean_health_popular = mean_health_at_k(top_n_popular, recipes, k=10)
mean_health_user_mean = mean_health_at_k(top_n_user_mean, recipes, k=10)
mean_health_svd = mean_health_at_k(top_n, recipes, k=10) 

print(f"MeanHealth@10 Random: {mean_health_random:.4f}")
print(f"MeanHealth@10 Most Popular: {mean_health_popular:.4f}")
print(f"MeanHealth@10 User Mean: {mean_health_user_mean:.4f}")
print(f"MeanHealth@10 SVD: {mean_health_svd:.4f}")

MeanHealth@10 Random: 3.3000
MeanHealth@10 Most Popular: 1.5258
MeanHealth@10 User Mean: 2.4859
MeanHealth@10 SVD: 2.4806
